### ***Step 1: Import Libraries***

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50, MobileNetV3Small
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import cv2
from glob import glob
import shutil


### ***Step 2: Load and Organize Data***

In [ ]:
# Define dataset paths
dataset_path = "path/to/dataset"  # Update with your dataset path
train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

# Define class labels
class_labels = ["cricket", "tennis", "badminton", "swimming"]

# Print dataset structure
for label in class_labels:
    print(f"{label}: {len(os.listdir(os.path.join(train_path, label)))} images")


### ***Step 3: Data Preprocessing***

In [ ]:
# Image parameters
IMG_SIZE = (224, 224)  # Resize images
BATCH_SIZE = 32

# Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.3
)

# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Test data generator (without augmentation)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode=None,
    shuffle=False
)


### ***Step 4: Define Deep Neural Network (DNN) Model***

In [ ]:
def build_dnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2,2),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

dnn_model = build_dnn()
dnn_model.summary()


### ***Step 5: Train DNN Model***

In [ ]:
history_dnn = dnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


### ***Step 6: Load Pretrained ResNet50 Model***

In [ ]:
def build_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze layers
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

resnet50_model = build_resnet50()
resnet50_model.summary()


### ***Step 7: Train ResNet50 Model***


In [ ]:
history_resnet50 = resnet50_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


### ***Step 8: Load Pretrained MobileNetV3 Model***

In [ ]:
def build_mobilenetv3():
    base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze layers
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

mobilenetv3_model = build_mobilenetv3()
mobilenetv3_model.summary()


### ***Step 7: Train MobileNetV3 Model***


In [ ]:
history_mobilenetv3 = mobilenetv3_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


### ***Step 10: Evaluate Models***

In [ ]:
def evaluate_model(model, model_name):
    y_pred = np.argmax(model.predict(val_generator), axis=1)
    y_true = val_generator.classes
    print(f"Classification Report for {model_name}:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix for {model_name}")
    plt.show()

# Evaluate each model
evaluate_model(dnn_model, "DNN")
evaluate_model(resnet50_model, "ResNet50")
evaluate_model(mobilenetv3_model, "MobileNetV3")


### ***Step 11: Plot Accuracy & Loss Graphs***

In [ ]:
def plot_history(history, model_name):
    # Accuracy plot
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f"{model_name} Accuracy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f"{model_name} Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

# Plot for each model
plot_history(history_dnn, "DNN")
plot_history(history_resnet50, "ResNet50")
plot_history(history_mobilenetv3, "MobileNetV3")
